### Step 1: Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import joblib
import cv2
import mediapipe as mp
import random
import warnings
warnings.filterwarnings("ignore") 

### Step 2: Load extracted hand landmark dataset

In [2]:
df = pd.read_csv('landmark_data.csv')
df.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,label
0,0.478694,0.746527,-5.949228e-07,0.578396,0.679919,-0.045107,0.646573,0.546933,-0.057510,0.654997,...,0.380448,0.475005,-0.090110,0.418307,0.552350,-0.078464,0.427703,0.598037,-0.054155,A
1,0.419936,0.629200,-5.375680e-07,0.519622,0.588439,-0.034311,0.589386,0.518284,-0.039314,0.630074,...,0.366136,0.376227,-0.017717,0.369763,0.424943,-0.013982,0.381743,0.462852,0.002650,A
2,0.487334,0.871052,-1.117565e-06,0.624713,0.782329,-0.043476,0.726665,0.668082,-0.060560,0.761545,...,0.384625,0.495017,-0.074173,0.396679,0.590808,-0.054504,0.404049,0.671423,-0.021249,A
3,0.466511,0.674685,-7.362099e-07,0.578875,0.576514,-0.039503,0.647641,0.420440,-0.051463,0.667817,...,0.364000,0.276959,-0.075096,0.391577,0.370270,-0.059260,0.400247,0.436620,-0.032790,A
4,0.500365,0.675875,-7.010962e-07,0.441767,0.624624,-0.000903,0.415099,0.549763,-0.001296,0.418380,...,0.534543,0.480551,-0.028805,0.525306,0.526025,-0.016429,0.533727,0.552502,-0.002215,A


### Step 3: Data Cleaning

In [3]:
df.isnull().sum()

x0       0
y0       0
z0       0
x1       0
y1       0
        ..
z19      0
x20      0
y20      0
z20      0
label    0
Length: 64, dtype: int64

### Step 4: Split dataset into features and labels

In [4]:
X = df.drop('label', axis=1)
y = df['label']

### Step 5: Encode class labels as integers

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
label_encoder.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
      dtype=object)

### Step 6: Normalize the feature values using StandardScaler

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Step 7: Split the data into training and testing sets (75% train, 25% test)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.25, random_state=42
)
print("Train:", len(X_train), "Test:", len(X_test))

Train: 1971 Test: 658


### Step 8: Model Building

#### MLP Classifier

In [8]:
# Initialize a simple MLP model with one hidden layer
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Train the model using the training dataset
model.fit(X_train, y_train)

# Predict the labels for the test dataset
y_pred = model.predict(X_test)

# Evaluate the model's accuracy on test data
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", round(acc * 100, 2), "%")

# Display detailed classification metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 97.11 %

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       0.91      0.95      0.93        22
           2       0.91      0.91      0.91        33
           3       0.95      0.95      0.95        19
           4       1.00      0.97      0.99        36
           5       0.96      0.96      0.96        24
           6       1.00      0.92      0.96        26
           7       1.00      1.00      1.00        28
           8       0.91      1.00      0.95        21
           9       1.00      1.00      1.00        24
          10       0.94      1.00      0.97        33
          11       1.00      1.00      1.00        24
          12       0.96      0.92      0.94        26
          13       0.95      0.95      0.95        22
          14       1.00      1.00      1.00        22
          15       1.00      1.00      1.00        27
          16       1.00      1.00      

#### Random Forest Classifier

In [9]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Accuracy:", round(accuracy_score(y_test, y_pred_rf) * 100, 2), "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 95.29 %

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.95      0.93        20
           1       1.00      1.00      1.00        22
           2       0.84      0.97      0.90        33
           3       1.00      0.89      0.94        19
           4       0.94      0.94      0.94        36
           5       0.96      0.92      0.94        24
           6       1.00      0.96      0.98        26
           7       0.96      0.96      0.96        28
           8       0.95      0.95      0.95        21
           9       1.00      1.00      1.00        24
          10       0.97      1.00      0.99        33
          11       0.92      1.00      0.96        24
          12       0.91      0.81      0.86        26
          13       1.00      0.91      0.95        22
          14       0.96      1.00      0.98        22
          15       1.00      1.00      1.00        27
          16       1.00      1.00     

#### Convolutional Neural Network (CNN)

In [10]:
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Reshape the input features to 3D as required by CNN input
X_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# One-hot encode labels for multi-class classification
y_cnn = to_categorical(y_encoded)

# Use previously split data (reshape train and test inputs)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)

# Define a 1D Convolutional Neural Network architecture
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train_cnn.shape[1], activation='softmax'))

# Compile the model with appropriate loss function and optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model and validate on test data
history = model.fit(X_train_cnn, y_train_cnn, epochs=30, batch_size=16, validation_data=(X_test_cnn, y_test_cnn))

# Evaluate model predictions and print performance metrics
y_pred_probs = model.predict(X_test_cnn)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_cnn, axis=1)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1495 - loss: 3.0212 - val_accuracy: 0.6702 - val_loss: 1.3577
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6280 - loss: 1.3681 - val_accuracy: 0.8343 - val_loss: 0.6436
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7765 - loss: 0.8225 - val_accuracy: 0.8632 - val_loss: 0.4741
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8355 - loss: 0.6050 - val_accuracy: 0.8997 - val_loss: 0.3608
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8705 - loss: 0.4561 - val_accuracy: 0.9179 - val_loss: 0.2917
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8921 - loss: 0.3724 - val_accuracy: 0.9210 - val_loss: 0.2481
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9038 - loss: 0.3578 - val_accuracy: 0.9362 - val_loss: 0.2490
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9095 - loss: 0.3011 - val_accuracy: 0.

#### LightGBM

In [11]:
# Flatten CNN input shape back to 2D for classical models
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Directly use y_train and y_test as they are already 1D (integer labels)
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_train_flat, y_train)
y_pred_lgbm = lgbm.predict(X_test_flat)

# Evaluate model
print("Accuracy:", round(accuracy_score(y_test, y_pred_lgbm) * 100, 2), "%")
print("Classification Report:\n", classification_report(y_test, y_pred_lgbm))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16065
[LightGBM] [Info] Number of data points in the train set: 1971, number of used features: 63
[LightGBM] [Info] Start training from score -3.204270
[LightGBM] [Info] Start training from score -3.229587
[LightGBM] [Info] Start training from score -3.381604
[LightGBM] [Info] Start training from score -3.143645
[LightGBM] [Info] Start training from score -3.411909
[LightGBM] [Info] Start training from score -3.229587
[LightGBM] [Info] Start training from score -3.155480
[LightGBM] [Info] Start training from score -3.282231
[LightGBM] [Info] Start training from score -3.427413
[LightGBM] [Info] Start training from score -3.255563
[LightGBM] [Info] Start training from score -3.323616
[LightGBM] [Info] Start training from score -3.242491

#### XGBoost

In [12]:
# Instantiate the XGBoost classifier
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')

# Fit the model on training data
xgb.fit(X_train_flat, y_train)

# Predict on test data
y_pred_xgb = xgb.predict(X_test_flat)

# Evaluate performance
print("Accuracy:", round(accuracy_score(y_test, y_pred_xgb) * 100, 2), "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))

Accuracy: 94.22 %

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        20
           1       0.91      0.95      0.93        22
           2       0.94      0.94      0.94        33
           3       0.90      1.00      0.95        19
           4       0.92      0.94      0.93        36
           5       0.91      0.88      0.89        24
           6       0.92      0.85      0.88        26
           7       1.00      0.96      0.98        28
           8       0.95      0.86      0.90        21
           9       1.00      0.96      0.98        24
          10       1.00      0.97      0.98        33
          11       1.00      1.00      1.00        24
          12       0.88      0.88      0.88        26
          13       0.95      0.86      0.90        22
          14       0.92      1.00      0.96        22
          15       1.00      1.00      1.00        27
          16       0.97      0.97     

#### Ensemble Learning using Stacking (Selected model)

In [13]:
# Define base learners for stacking
base_learners = [
    ('rf', RandomForestClassifier(random_state=42)),  # Random Forest
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),  # XGBoost
    ('mlp', MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42))  # MLP
]

# Logistic Regression as meta-learner (final estimator)
meta_learner = LogisticRegression()

# Build the stacking ensemble
stack_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    passthrough=True  # includes original features along with base predictions
)

# Train the stacking model
stack_model.fit(X_train_flat, y_train)

# Predict and evaluate
y_pred = stack_model.predict(X_test_flat)
print("Accuracy:", round(accuracy_score(y_test, y_pred) * 100, 2), "%")

Accuracy: 98.02 %


## Step 9: Save Final Trained Model for Deployment

In [14]:
joblib.dump(stack_model, 'final_stack_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Model, label encoder, and scaler saved successfully.")

Model, label encoder, and scaler saved successfully.
